In [1]:
import numpy as np
from scipy.linalg import block_diag
import time

In [2]:
def I(L, i):
    card = int(n / (2**L))
    start = i * card 
    end = start + card 
    return start, end

In [3]:
def get(L, i, j=-1):
    #assert i < 2**L
    if j >= 0:
        #assert j < 2**L
        return int((4**L - 4)/3) + i * (2**L) + j
    else:
        return (2**L - 1) + i - 1

In [4]:
def generate_mat(params):
    mat = []
    for i in range(n):
        tmpv = np.zeros([n])
        tmpv[i] = 1
        colv, _ = hss(params, tmpv)
        mat.append(colv)
    return np.column_stack(mat)

In [5]:
def hss(params, vec):
    u, v, x, y, s, k, p, mat = params
    res = np.zeros([n])
    tmpX = np.zeros([2**(p+1)-2, k])
    tmpY = np.zeros([2**(p+1)-2, k])
    
    start = time.time()
    # step 1
    L = p
    for i in range(2**L):
        si, ei = I(L, i)
        tmpX[get(L, i)] = v[i].T.dot(vec[si:ei])

    # step 2
    for L in range(p-1, 0, -1):
        for i in range(2**L):
            tmpX[get(L,i)] = y[get(L,i)].T.dot(
                np.hstack([tmpX[get(L+1, 2*i)], tmpX[get(L+1, 2*i+1)]]))

    # step 3
    for L in range(1, p):
        for i in range(2**(L-1)):
            tmpY[get(L,2*i)] = s[get(L,2*i,2*i+1)].dot(tmpX[get(L,2*i+1)])
            tmpY[get(L,2*i+1)] = s[get(L,2*i+1,2*i)].dot(tmpX[get(L,2*i)])

    # step 4
    for L in range(1, p):
        for i in range(2**L):
            tmp = x[get(L, i)].dot(tmpY[get(L,i)])
            tmpY[get(L+1,2*i)] += tmp[:k]
            tmpY[get(L+1,2*i+1)] += tmp[k:]

    # step 5
    L = p
    for i in range(2**L):
        si, ei = I(L, i)
        res[si:ei] = u[i].dot(tmpY[get(L,i)]) + mat[si:ei,si:ei].dot(vec[si:ei])
    
    end = time.time()
    return res, (end - start)*1000

In [6]:
p = 8
n0 = 3
k = 1 # rank 
n = (2**p) * n0 # matrix size
print("p={}, n={}".format(p,n))

u = np.random.random([2**p, n0, k])  # O(nk)
v = np.random.random([2**p, n0, k])  # O(nk)
x = np.random.random([2**p-2, 2*k, k])  # O(pk^2)
y = np.random.random([2**p-2, 2*k, k])  # O(pk^2)
size = (4**p-4)/3
s = np.random.random([int(size), k, k])  # O(nk^2/n0)
print(size)
print("u", u.shape)
print("v", v.shape)
print("x", x.shape)
print("y", y.shape)
print("s", s.shape)

p=8, n=768
21844.0
u (256, 3, 1)
v (256, 3, 1)
x (254, 2, 1)
y (254, 2, 1)
s (21844, 1, 1)


In [7]:
# build up mat
# fill up the diagonal blocks
blockmat = np.zeros([n, n])
L = p
blocksize = n0 * 2**(p-L)
for i in range(2**L):
    si, ei = I(L, i)
    blockmat[si:ei, si:ei] = np.random.random([blocksize, blocksize])
    
pa = (u, v, x, y, s, k, p, blockmat)
mat = generate_mat(pa)

In [8]:
vec = np.random.random([n])

In [9]:
repeat = 10

In [10]:
start = time.time()
for _ in range(repeat):
    ref = mat.dot(vec)
end = time.time()
print((end - start)*1000 / repeat, "ms")

11.230134963989258 ms


In [11]:
ref2 = np.zeros([n])
start = time.time()
for _ in range(repeat):
    for i in range(n):
        ref2[i] = 0
        for j in range(n):
            ref2[i] += mat[i,j]*vec[j]
end = time.time()
print((end - start)*1000 / repeat, "ms")

685.2351427078247 ms


In [12]:
t = 0
for _ in range(repeat):
    ours, tmpt = hss(pa, vec)
    t += tmpt
print(t / repeat, "ms")

11.218070983886719 ms


In [13]:
print(np.allclose(ref, ref2), np.allclose(ref, ours))

True True
